<a href="https://colab.research.google.com/github/Udit-dutta/Udit-dutta/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [5]:
import os
import json
import zipfile
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

DATA COLLECTION

In [6]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [9]:
os.environ['KAGGLE_USERNAME']=kaggle_dictionary["username"]
os.environ['KAGGLE_KEY']=kaggle_dictionary["key"]

In [10]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 93% 24.0M/25.7M [00:01<00:00, 27.2MB/s]
100% 25.7M/25.7M [00:01<00:00, 16.6MB/s]


In [11]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [12]:
with ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip','r') as zip:
  zip.extractall()
  print("Extracted")


Extracted


Loading the dataset and preprocessing it.


In [14]:
data=pd.read_csv("/content/imdb-dataset-of-50k-movie-reviews.zip")
data.shape

(50000, 2)

In [15]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [16]:
# check the uniformity of the dataset
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [19]:
data.replace({"sentiment":{"positive" : 1,"negative" : 0}}, inplace = True)

In [20]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [21]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [22]:
train_data,test_data=train_test_split(data,test_size=0.2, random_state=42)

In [23]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


DATA PREPROCESSING

In [25]:
tokenizer= Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
x_train_data= pad_sequences(tokenizer.texts_to_sequences(train_data["review"]),maxlen=200)
x_test_data= pad_sequences(tokenizer.texts_to_sequences(test_data["review"]),maxlen=200)

In [26]:
y_train_data=train_data["sentiment"]
y_test_data=test_data["sentiment"]

BUILDING LSTM MODEL

In [27]:
model=Sequential()
model.add(Embedding(input_dim=5000,output_dim=128,input_length=200))
model.add(LSTM(128,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1,activation="sigmoid"))


In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          640000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
#compil
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [31]:
 model.fit(x_train_data,y_train_data,epochs=5,batch_size=64,validation_split=0.2)

Epoch 1/5
500/500 [==============================] - 435s 862ms/step - loss: 0.4114 - accuracy: 0.8122 - val_loss: 0.3274 - val_accuracy: 0.8627
Epoch 2/5
500/500 [==============================] - 406s 813ms/step - loss: 0.2656 - accuracy: 0.8941 - val_loss: 0.3129 - val_accuracy: 0.8691
Epoch 3/5
500/500 [==============================] - 387s 775ms/step - loss: 0.2302 - accuracy: 0.9090 - val_loss: 0.3280 - val_accuracy: 0.8669
Epoch 4/5
500/500 [==============================] - 390s 780ms/step - loss: 0.1960 - accuracy: 0.9247 - val_loss: 0.3314 - val_accuracy: 0.8624
Epoch 5/5
500/500 [==============================] - 389s 779ms/step - loss: 0.1696 - accuracy: 0.9355 - val_loss: 0.3833 - val_accuracy: 0.8740


MODEL EVALUATION

In [33]:
loss,accuracy=model.evaluate(x_test_data,y_test_data)
print(f"Test accuracy:{accuracy}")

313/313 [==============================] - 28s 89ms/step - loss: 0.3611 - accuracy: 0.8806
Test accuracy:0.8805999755859375


BUILDING A PREDICTIVE SYSTEM

In [34]:
def predict_sentiment(review):
  sequence=tokenizer.texts_to_sequences(review)
  padded_sequence=pad_sequences(sequence,maxlen=200)
  prediction=model.predict(padded_sequence)
  sentiment="Positive" if prediction[0]>0.5 else "Negative"
  return sentiment

In [41]:
new_review = input("")
sentiment=predict_sentiment(new_review)
print(f"The sentiment of the review is:{sentiment}")

Was drawn into seeing 'Negative', with a cool poster/cover, a promising trailer, an intriguing premise and as someone with a general appreciation for drama/thrillers. That it was low-budget, which from frequent personal experience is rarely a good sign due to that there are so many poor ones out there, made me though apprehensive.
11/11 [==============================] - 2s 134ms/step
The sentiment of the review is:Negative
